In [70]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import *
import pandas as pd
%matplotlib inline

In [18]:
X = np.array([[0.4005,0.5306],
             [0.2148,0.3854],
             [0.3457,0.3156],
             [0.2652,0.1875],
             [0.0789,0.4139],
             [0.4548,0.3022]])

### Getting the Similarity Table

In [111]:
Similarity_Table =[]
for P1 in range(len(X)):
    temp = []
    for P2 in range(len(X)):
        if(P1 ==P2):
            temp.append(-0.153778)
        else:
            temp.append(-1*((X[P1][0]-X[P2][0])*(X[P1][0]-X[P2][0]) + (X[P1][1]-X[P2][1])*(X[P1][1]-X[P2][1])))
    if(len(temp) !=0):
        Similarity_Table.append(temp)


In [112]:
Similarity_Table = np.array(Similarity_Table)

In [113]:
df = pd.DataFrame(Similarity_Table)
df

,0,1,2,3,4,5
0,-0.153778,-0.055568,-0.049228,-0.136024,-0.117045,-0.055115
1,-0.055568,-0.153778,-0.022007,-0.041705,-0.019281,-0.064522
2,-0.049228,-0.022007,-0.153778,-0.022890,-0.080845,-0.012082
3,-0.136024,-0.041705,-0.022890,-0.153778,-0.085965,-0.049104
4,-0.117045,-0.019281,-0.080845,-0.085965,-0.153778,-0.153778
5,-0.055115,-0.064522,-0.012082,-0.049104,-0.153778,-0.153778


In [101]:
Similarity_Table.reshape(6,6)

array([[-0.153778  , -0.05556753, -0.04922804, -0.1360237 , -0.11704545,
        -0.05511505],
       [-0.05556753, -0.153778  , -0.02200685, -0.04170457, -0.01928106,
        -0.06452224],
       [-0.04922804, -0.02200685, -0.153778  , -0.02288986, -0.08084513,
        -0.01208237],
       [-0.1360237 , -0.04170457, -0.02288986, -0.153778  , -0.08596465,
        -0.04910425],
       [-0.11704545, -0.01928106, -0.08084513, -0.08596465, -0.153778  ,
        -0.1537777 ],
       [-0.05511505, -0.06452224, -0.01208237, -0.04910425, -0.1537777 ,
        -0.153778  ]])

In [102]:
Similarity_Table.shape

(6, 6)

In [143]:
x = np.array([5,6,2,2])
y = np.zeros(len(x))
t = np.amax((x,y),axis = 1)
t

array([6., 0.])

## Do the Algorithm to Calculate The Responsibility and Availability.

In [418]:
def AffanityProp(s,damping,do_dumping):

    n = len(s)
    a = np.zeros((n,n))
    r = np.zeros((n,n))
    
    for num_iterations in range(5):
        r_old = r.copy()
        a_old = a.copy()
        for i in range(n):
            for k in range(n):
                
                temp_s = np.hstack((s[i,:k],s[i,k+1:]))
                temp_a = np.hstack((a[i,:k],a[i,k+1:]))
                r[i,k] = s[i,k] - np.amax(temp_s + temp_a)

        #Fill availibility 

        for i in range(n):
            for k in range(n):
                if(k == i):
                    t_a = np.hstack((r[:i,i],r[i+1:,i]))
                    zero = np.zeros(len(t_a))
                    a[i,i] = sum(np.maximum(zero,t_a))
                    continue

                a_mx = np.hstack((r[:i,k],r[i+1:,k]))
                zero = np.zeros(len(a_mx))
                a_mx = np.maximum(a_mx,zero)
                a[i,k] = min(0,r[k,k] + sum(a_mx))
        
        if(do_dumping):
            if(num_iterations !=0):
                r = damping*r_old+ (1-damping)*r
                a = damping*a_old+ (1-damping)*a
           
    
    return a,r

In [419]:
a,r = AffanityProp(Similarity_Table,0.8,1)

In [420]:
rdf = pd.DataFrame(r)
rdf


,0,1,2,3,4,5
0,-0.070934,0.028620,0.036027,-0.053180,-0.034201,0.027729
1,-0.040341,-0.100131,-0.006780,-0.026478,-0.002938,-0.049296
2,-0.038376,-0.011155,-0.105963,-0.012038,-0.069993,0.002835
3,-0.073631,0.021509,0.048688,-0.091385,-0.023572,0.013289
4,-0.042971,0.090573,-0.006771,-0.011890,-0.079407,-0.079703
5,0.003408,-0.005999,0.064457,0.009419,-0.095255,-0.093775


In [421]:
adf = pd.DataFrame(a)
adf


,0,1,2,3,4,5
0,0.033896,-0.029873,-0.035168,-0.066637,-0.076454,-0.070229
1,-0.055082,0.173659,-0.032757,-0.066637,-0.077580,-0.057901
2,-0.055082,-0.029873,0.174808,-0.066637,-0.076454,-0.061966
3,-0.055082,-0.029873,-0.040463,0.052261,-0.078281,-0.065924
4,-0.055082,-0.057886,-0.032757,-0.070983,0.015678,-0.057901
5,-0.067104,-0.029873,-0.047921,-0.078619,-0.076454,0.069891


In [422]:
c = r + a
cdf = pd.DataFrame(c)
cdf

,0,1,2,3,4,5
0,-0.037038,-0.001253,0.000860,-0.119816,-0.110656,-0.042500
1,-0.095423,0.073528,-0.039537,-0.093115,-0.080518,-0.107197
2,-0.093458,-0.041028,0.068845,-0.078675,-0.146448,-0.059131
3,-0.128713,-0.008364,0.008225,-0.039124,-0.101853,-0.052635
4,-0.098053,0.032687,-0.039527,-0.082873,-0.063729,-0.137604
5,-0.063696,-0.035872,0.016536,-0.069200,-0.171709,-0.023884


In [423]:
ClusterPoints = np.zeros(len(c))
for row in range(len(c)):
    ClusterPoints[row] = np.argmax(c[row,])

In [424]:
ClusterPoints

array([2., 1., 2., 2., 1., 2.])

In [425]:
a,r = AffanityProp(Similarity_Table,0.8,0)

In [426]:
rdf = pd.DataFrame(r)
rdf


,0,1,2,3,4,5
0,-0.104550,-0.239971,-0.233632,-0.320428,-0.301449,-0.239519
1,-0.467597,-0.134497,-0.434037,-0.453734,-0.431311,-0.476552
2,-0.467061,-0.439840,-0.141696,-0.440723,-0.498678,-0.429915
3,-0.397720,-0.303401,-0.284586,-0.130888,-0.347661,-0.310801
4,-0.257515,-0.159751,-0.221315,-0.226434,-0.134497,-0.294247
5,-0.317897,-0.327304,-0.274864,-0.311886,-0.416560,-0.141696


In [427]:
adf = pd.DataFrame(a)
adf


,0,1,2,3,4,5
0,0.00000,-0.134497,-0.141696,-0.130888,-0.134497,-0.141696
1,-0.10455,0.000000,-0.141696,-0.130888,-0.134497,-0.141696
2,-0.10455,-0.134497,0.000000,-0.130888,-0.134497,-0.141696
3,-0.10455,-0.134497,-0.141696,0.000000,-0.134497,-0.141696
4,-0.10455,-0.134497,-0.141696,-0.130888,0.000000,-0.141696
5,-0.10455,-0.134497,-0.141696,-0.130888,-0.134497,0.000000


In [428]:
c = r + a
cdf = pd.DataFrame(c)
cdf

,0,1,2,3,4,5
0,-0.104550,-0.374468,-0.375328,-0.451316,-0.435946,-0.381215
1,-0.572147,-0.134497,-0.575732,-0.584622,-0.565808,-0.618248
2,-0.571611,-0.574336,-0.141696,-0.571611,-0.633175,-0.571611
3,-0.502270,-0.437898,-0.426282,-0.130888,-0.482158,-0.452497
4,-0.362065,-0.294248,-0.363010,-0.357322,-0.134497,-0.435943
5,-0.422447,-0.461801,-0.416560,-0.442774,-0.551057,-0.141696


In [430]:
ClusterPoints = np.zeros(len(c))
for row in range(len(c)):
    ClusterPoints[row] = np.argmax(c[row,])
ClusterPoints

array([0., 1., 2., 3., 4., 5.])